In [ ]:
import spacy
from fuzzywuzzy import process
import mysql.connector
import openai
from fpdf import FPDF
from fpdf.enums import XPos, YPos
from io import BytesIO

#!!!SkillsMatcher!!!
# Load SpaCy model
nlp = spacy.load("en_core_web_sm")

# Sample job description
job_description = """
Junior Software Engineer – Full Stack
About Us:
Join our forward-thinking startup, where you will work on innovative projects and develop both your front-end and back-end skills. We are looking for passionate freshers eager to make a mark in the tech industry.
Key Responsibilities:
•	Develop and maintain web applications, working on both front-end and back-end components.
•	Collaborate with designers and other developers to create user-friendly interfaces.
•	Write clean, modular, and scalable code.
•	Troubleshoot and debug issues across the entire stack.
•	Participate in code reviews and contribute to best practices.
Skills:
•	Proficiency in HTML, CSS, JavaScript, and a front-end framework (React, Angular, or Vue.js).
•	Experience with server-side languages (Node.js, Python, Ruby, or Java).
•	Understanding of RESTful APIs and web services.
•	Familiarity with databases (SQL and NoSQL).
•	Basic knowledge of version control systems (Git).
•	Strong problem-solving skills.
•	Effective communication and teamwork abilities.
•	Ability to manage time efficiently.
•	Willingness to learn and adapt to new technologies.
Qualifications:
•	Pursuing or completed a degree in Computer Science, IT, Computer Science and Engineering, or related fields.
•	Internship experience in full stack development is a plus.
Deadline: 26th June

"""
    
def extract_and_match_skills(job_description, skills_list, threshold=80):
    
    # Process job description with SpaCy
    doc = nlp(job_description)

    # Extract skills as noun phrases and named entities
    extracted_skills = set()
    for chunk in doc.noun_chunks:
        lemma_chunk = " ".join([token.lemma_ for token in chunk])
        if len(lemma_chunk) > 1:  # Filter out single letters
            extracted_skills.add(lemma_chunk.lower().strip())

    for entity in doc.ents:
        lemma_entity = " ".join([token.lemma_ for token in entity])
        if len(lemma_entity) > 1:  # Filter out single letters
            extracted_skills.add(lemma_entity.lower().strip())

    # Perform fuzzy matching
    matches = []
    for skill in skills_list:
        match, score = process.extractOne(skill, extracted_skills)
        if score >= threshold:
            matches.append(skill)
    
    return matches
    
#!!!QuestionsGenerator!!!

# Set up OpenAI API key directly
openai.api_key = ''  

# Function to create a prompt for generating interview questions
def create_prompt(skills):
    prompt = "Generate personalized interview questions for any five of the following skills:\n"
    for skill in skills:
        prompt += f"- {skill}\n"
    prompt += "\nFor each skill, create a detailed and relevant question that a hiring manager might ask a candidate during an interview. No need for explanation"
    return prompt

# Generate interview questions using OpenAI
def generate_questions(candidate_skills):
    if not candidate_skills:
        return[]

    try: 
        # Create the prompt
        prompt = create_prompt(candidate_skills)
        
        # Generate questions using the OpenAI API
        response = openai.ChatCompletion.create(
            model="gpt-3.5-turbo",  # Replace with the appropriate model name
            messages=[
                {"role": "system", "content": "You are a helpful assistant."},
                {"role": "user", "content": prompt}
            ],
            max_tokens=200,  # Adjust as needed to ensure the questions are complete
            n=1,
            temperature=0.7,
        )
        
        # Extract and print the generated questions
        generated_text = response.choices[0].message['content'].strip()
        generated_questions = generated_text.split('\n')
        return generated_questions

    except Exception as e:
        print(f"Error generating questions: {e}")
        return []  # Return an empty list if there is an error

#!!!PDF Creator!!!
# Custom PDF class with header and footer
class CustomPDF(FPDF):
    def header(self):
        self.set_font("helvetica", "B", 14)
        self.cell(0, 10, "SMARTRECRUITER - X DOCUMENT", new_x=XPos.LMARGIN, new_y=YPos.NEXT, align='C')
        self.set_text_color(0, 0, 255)
        self.set_draw_color(0, 0, 128)
        self.set_line_width(0.5)
        self.line(10, 20, 200, 20)  
        self.ln(10)

    def footer(self):
        self.set_y(-15)
        self.set_font("helvetica", "I", 10)
        self.set_text_color(128)
        page_number_text = f"Page {self.page_no()} of {{nb}}"
        self.cell(0, 10, page_number_text, align="C")

# Function to create PDF in memory
def create_pdf(name, GeneratedQuestions):
    pdf = CustomPDF()
    pdf.alias_nb_pages()
    pdf.add_page()

    # Add the main heading with a larger font size
    pdf.set_font("helvetica", "B", size=36)  # Bold, size 36
    pdf.set_text_color(0, 0, 255)
    pdf.cell(200, 10, text="Recommended Questions", new_x=XPos.LMARGIN, new_y=YPos.NEXT, align='C')
    pdf.ln(5)  # Space after heading

    # Reset text color if needed for other parts
    pdf.set_text_color(0, 0, 0)  # Back to black for other text
    
    # Add a subheading with a slightly smaller font size
    pdf.set_font("helvetica", "I", size=20)  # Italic, size 20
    pdf.cell(200, 10, text=f"Candidate Name: {name}", new_x=XPos.LMARGIN, new_y=YPos.NEXT, align='C')
    pdf.ln(10)  # Space after subheading

    # Add questions
    pdf.set_font("helvetica", size=14)
    cell_width = 190
    line_height = 6

    for line in GeneratedQuestions:
        pdf.multi_cell(cell_width, line_height, text=line)
        pdf.ln(3)

    # Save PDF to a BytesIO object (in-memory buffer)
    pdf_buffer = BytesIO()
    pdf.output(pdf_buffer)
    pdf_buffer.seek(0)  # Go back to the beginning of the buffer
    return pdf_buffer.getvalue()


# Function to save PDF and generated questions to the database
def save_to_db(pdf_data, questions, email):
    try:
        conn = mysql.connector.connect(
            host='localhost',
            user='root',
            password='',
            database='interviewsupportsystem'
        )
        cursor = conn.cursor()
        cursor.execute("""
            UPDATE resumes SET Questions = %s WHERE C_Email = %s
        """, (pdf_data, email))
        conn.commit()
        print(f"PDF and questions for {email} saved successfully to database.")

    except mysql.connector.Error as err:
        print(f"Error: {err}")
    finally:
        if conn.is_connected():
            cursor.close()
            conn.close()

# Main function to fetch each candidate's skills, generate questions, and save to database
def process_candidates():
    try:
        conn = mysql.connector.connect(
            host='localhost',
            user='root',
            password='',
            database='interviewsupportsystem'
        )
        cursor = conn.cursor()
        cursor.execute("""
            SELECT user.First_Name, user.Last_Name, resumes.C_Email, resumes.Structured_Resume
            FROM resumes
            JOIN user ON resumes.C_Email = user.Email
        """)
        rows = cursor.fetchall()

        for row in rows:
            First_Name, Last_Name, C_Email, skills_list = row
            
            full_name = f"{First_Name} {Last_Name}"
            new_skills_list = [skill.strip() for skill in skills_list.split(",")]
            
            matched_skills = extract_and_match_skills(job_description, new_skills_list, threshold=80)
            generated_questions = generate_questions(matched_skills)
            if generated_questions:
                pdf_data = create_pdf(full_name, generated_questions)
                save_to_db(pdf_data, generated_questions, C_Email)
            else:
                print(f"No questions generated for {name}.")

    except mysql.connector.Error as err:
        print(f"Error: {err}")
    finally:
        if conn.is_connected():
            cursor.close()
            conn.close()

# Run the process
process_candidates()


PDF and questions for kixibop859@luravell.com saved successfully to database.
PDF and questions for kixibop859@luravell.com saved successfully to database.
PDF and questions for tojexeh940@mposhop.com saved successfully to database.
PDF and questions for wejacab336@nolanzip.com saved successfully to database.
